## Compute the camera calibration matrix and distortion coefficients

In [ ]:
# Source: https://github.com/udacity/CarND-Camera-Calibration/blob/master/camera_calibration.ipynb

import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

display_chessboard_corners = True
save_chessboard_corners = False

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        if display_chessboard_corners or save_chessboard_corners:
            cv2.drawChessboardCorners(img, (9,6), corners, ret)
        
        if display_chessboard_corners:
            cv2.imshow('img', img)
            cv2.waitKey(100)
            
        if save_chessboard_corners:
            write_name = 'output_images/corners_found' + str(idx) + '.jpg'
            cv2.imwrite(write_name, img)

if display_chessboard_corners or save_chessboard_corners:
    cv2.destroyAllWindows()

if objpoints and imgpoints:
    print('Success: Found object points and image points.')
else:
    assert False, 'Failure: Did not find object points and image points.'

In [ ]:
# Source: https://github.com/udacity/CarND-Camera-Calibration/blob/master/camera_calibration.ipynb

import pickle
%matplotlib inline

# Test undistortion on an image
img = cv2.imread('camera_cal/calibration1.jpg')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

dst = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('output_images/test_undistort.jpg', dst)

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump(dist_pickle, open("camera_cal/wide_dist_pickle.p", "wb"))

# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)

## Apply a distortion correction to raw images.

In [ ]:
# Make a list of calibration images
images = glob.glob('test_images/*.jpg')

f, axes = plt.subplots(len(images), 2, figsize=(16, 6 * len(images)))

undistorted = []

for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    axes[idx][0].imshow(img)
    axes[idx][0].set_title('Original Image', fontsize=20)
    axes[idx][1].imshow(dst)
    axes[idx][1].set_title('Undistorted Image', fontsize=20)
    undistorted.append(dst)

## Use color transforms, gradients, etc., to create a thresholded binary image.

In [ ]:
def get_sobel_x(gray, ksize):
    return cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=ksize)

def get_sobel_y(gray, ksize):
    return cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=ksize)

def get_scaled_sobel_x_y(gray, ksize):
    sobelx = get_sobel_x(gray, ksize)
    sobely = get_sobel_y(gray, ksize)
    import math
    magnitude = np.sqrt((sobelx ** 2) + (sobely ** 2))
    scaled_mag = np.uint8(255 * magnitude / np.max(magnitude))
    return scaled_mag

def abs_sobel_thresh(gray, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    if orient == 'x':
        sobel = get_sobel_x(gray, sobel_kernel)
    elif orient == 'y':
        sobel = get_sobel_y(gray, sobel_kernel)
    # Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sbinary

def mag_thresh(gray, sobel_kernel=3, thresh=(0, 255)):
    # Calculate gradient magnitude
    # Apply threshold
    scaled_mag = get_scaled_sobel_x_y(gray, sobel_kernel)
    # Create a binary mask where mag thresholds are met
    binary = np.zeros_like(scaled_mag)
    binary[(scaled_mag >= thresh[0]) & (scaled_mag <= thresh[1])] = 1
    return binary

# Define a function that applies Sobel x and y, 
# then computes the direction of the gradient
# and applies a threshold.
def dir_threshold(gray, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Take the gradient in x and y separately
    sobelx = get_sobel_x(gray, ksize=sobel_kernel)
    sobely = get_sobel_y(gray, ksize=sobel_kernel)
    # Take the absolute value of the x and y gradients
    abs_sobelx = np.abs(sobelx)
    abs_sobely = np.abs(sobely)
    # Calculate the direction of the gradient 
    direction = np.arctan2(abs_sobely, abs_sobelx)
    # Create a binary mask where direction thresholds are met
    sbinary = np.zeros_like(direction)
    sbinary[(direction >= thresh[0]) & (direction <= thresh[1])] = 1
    return sbinary

# Define a function that thresholds the S-channel of HLS
# Use exclusive lower bound (>) and inclusive upper (<=)
def hls_select(img, h_thresh=(0, 180), l_thresh=(0, 255), s_thresh=(0, 255)):
    # Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    # Apply thresholds to the channels
    h_binary = np.zeros_like(H)
    h_binary[(H > h_thresh[0]) & (H <= h_thresh[1])] = 1
    l_binary = np.zeros_like(L)
    l_binary[(L > l_thresh[0]) & (L <= l_thresh[1])] = 1
    s_binary = np.zeros_like(S)
    s_binary[(S > s_thresh[0]) & (S <= s_thresh[1])] = 1
    return (h_binary, l_binary, s_binary)

class Thresholds:
    def __init__(self, sobel, mag, dir, h, l, s):
        self.sobel = sobel
        self.mag = mag
        self.dir = dir
        self.h = h
        self.l = l
        self.s = s
        
thresholds = Thresholds(sobel = (60, 140), mag = (80, 120), dir = (0.9, 1.1), h = (18, 25), l = (200, 255), s = (80, 255))

# Assumes input image is RGB.
def get_binary_image(img, sobel_kernel, thresh):    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=sobel_kernel, thresh=thresh.sobel)
    grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=sobel_kernel, thresh=thresh.sobel)
    mag_binary = mag_thresh(gray, sobel_kernel=sobel_kernel, thresh=thresh.mag)
    dir_binary = dir_threshold(gray, sobel_kernel=sobel_kernel, thresh=thresh.dir)
    
    (h_bin, l_bin, s_bin) = hls_select(img, thresh.h, thresh.l, thresh.s)
    
    h_s = np.zeros_like(h_bin)
    h_s[(h_bin == 1) & (s_bin == 1)] = 1
    l_s = np.zeros_like(l_bin)
    l_s[(l_bin == 1) & (s_bin == 1)] = 1
    mag_dir = np.zeros_like(mag_binary)
    mag_dir[(mag_binary == 1) & (dir_binary == 1)] = 1
    gradxy = np.zeros_like(gradx)
    gradxy[(gradx == 1) & (grady == 1)] = 1
    
    b1 = np.zeros_like(h_s)
    b2 = np.zeros_like(h_s)
    b1[(h_s == 1) | (l_s == 1) | (mag_dir == 1) | (gradxy == 1)] = 1
    b2[(h_s == 1) | (l_s == 1)] = 1
    return (b1, b2)

rows_per_img = 1

f, axes = plt.subplots(len(images) * rows_per_img, 3, figsize=(16, 4 * len(images)))

def show_image(axis, img, gray, title):
    if gray:
        axis.imshow(img, cmap='gray')
    else:
        axis.imshow(img)
    axis.set_title(title, fontsize=20)
    
binary = []

for idx, img in enumerate(undistorted):
    b = get_binary_image(img, 5, thresholds)
    binary.append(b[0])
    
    top = idx * rows_per_img
    mid = idx * rows_per_img + 1
    bottom = idx * rows_per_img + 2
    
    show_image(axes[top][0], img, False, 'Undistorted')
    show_image(axes[top][1], b[0], True, 'All')
    show_image(axes[top][2], b[1], True, '(H & S) | (L & S)')

## Apply a perspective transform to rectify binary image ("birds-eye view").

In [ ]:
img = cv2.imread('test_images/straight_lines1.jpg')
img_size = (img.shape[1], img.shape[0])

# Source for src and dst points: "example_writeup.pdf" included with provided project code.

src = np.float32(
    [[(img_size[0] / 2) - 62, img_size[1] / 2 + 100],
    [((img_size[0] / 6) - 10), img_size[1]],
    [(img_size[0] * 5 / 6) + 60, img_size[1]],
    [(img_size[0] / 2 + 65), img_size[1] / 2 + 100]])
dst = np.float32(
    [[(img_size[0] / 4), 0],
    [(img_size[0] / 4), img_size[1]],
    [(img_size[0] * 3 / 4), img_size[1]],
    [(img_size[0] * 3 / 4), 0]])

M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

def warp(img):
    return cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

f, axes = plt.subplots(len(images), 2, figsize=(20,30))

birds_eye = []

for idx, img in enumerate(binary):
    dst = warp(img)
    birds_eye.append(dst)
    show_image(axes[idx][0], img, True, 'Binary')
    show_image(axes[idx][1], dst, True, 'Bird\'s Eye')

## Detect lane pixels and fit to find the lane boundary.

In [ ]:
class Lane:
    def __init__(self, fit, curverad):
        self.fit = fit
        self.curverad = curverad
        
class Lanes:
    def __init__(self, left, right):
        self.left = left
        self.right = right

def find_lanes(binary_warped, axis):
    import numpy as np
    import cv2

    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    import matplotlib.pyplot as plt
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    if axis:
        axis.imshow(out_img)
        axis.plot(left_fitx, ploty, color='yellow')
        axis.plot(right_fitx, ploty, color='yellow')
        axis.set_title('Windows', fontsize = 20)
    
    (left_curverad, right_curverad) = calc_radius_of_curvature_meters(ploty, left_fitx, right_fitx)

    return Lanes(left = Lane(left_fit, left_curverad), right = Lane(right_fit, right_curverad))

def find_lanes_with_hint(binary_warped, prev_lanes, axis):
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    left_fit = prev_lanes.left.fit
    right_fit = prev_lanes.right.fit
    
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) 
                    & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) 
                     & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    if axis:
        axis.set_title('Hinted', fontsize=20)
        axis.imshow(result)
        axis.plot(left_fitx, ploty, color='yellow')
        axis.plot(right_fitx, ploty, color='yellow')
        #plt.xlim(0, 1280)
        #plt.ylim(720, 0)
        
    (left_curverad, right_curverad) = calc_radius_of_curvature_meters(ploty, left_fitx, right_fitx)

    return Lanes(left = Lane(left_fit, left_curverad), right = Lane(right_fit, right_curverad))

def calc_radius_of_curvature_meters(ploty, leftx, rightx):
    y_eval = np.max(ploty)
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    # print(left_curverad, 'm', right_curverad, 'm')
    # Example values: 632.1 m    626.2 m
    return (left_curverad, right_curverad)

f, axes = plt.subplots(len(images), 3, figsize=(20,30))

import statistics

def format_lanes(idx, lanes, title):
    print('{}, [{:.0f}, {:.0f}], Avg: [{:.0f}], {}'.format(idx, lanes.left.curverad, lanes.right.curverad, statistics.mean([lanes.left.curverad, lanes.right.curverad]), title))

for idx, img in enumerate(birds_eye):
    show_image(axes[idx][0], img, True, 'Binary')
    lanes = find_lanes(img, axes[idx][1])
    format_lanes(idx, lanes, 'non-hint')
    lanes_hint = find_lanes_with_hint(img, lanes, axes[idx][2])
    format_lanes(idx, lanes_hint, 'hint')

## Determine the curvature of the lane and vehicle position with respect to center.

## Warp the detected lane boundaries back onto the original image.

In [ ]:
def reverse_warp(warped, undist, lanes):
    left_fit = lanes.left.fit
    right_fit = lanes.right.fit
    
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, img_size) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result

f, axes = plt.subplots(len(images), 3, figsize=(20,30))

for idx, (binary_warped, undist) in enumerate(zip(birds_eye, undistorted)):
    show_image(axes[idx][0], binary_warped, True, 'Binary')
    lanes = find_lanes(binary_warped, None)
    lanes_hint = find_lanes_with_hint(binary_warped, lanes, axes[idx][1])
    
    reverse_warped = reverse_warp(binary_warped, undist, lanes)
    show_image(axes[idx][2], reverse_warped, False, 'Reverse')

## Incorporate reverse_warp in Pipeline.

In [ ]:
class PipelineResult:
    def __init__(self, original, undistorted, binary, birds_eye, lanes, reverse_warped):
        self.original = original
        self.undistorted = undistorted
        self.binary = binary
        self.birds_eye = birds_eye
        self.lanes = lanes
        self.reverse_warped = reverse_warped

class Pipeline:
    def __init__(self, mtx, dist, sobel_kernel, thresh):
        self.mtx = mtx
        self.dist = dist
        self.sobel_kernel = sobel_kernel
        self.thresh = thresh
        
    def run(self, fname, prev_result, axis):
        img = cv2.imread(fname)
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        undistorted = cv2.undistort(img, mtx, dist, None, mtx)
        (binary, ignored) = self.get_binary_image(undistorted)
        birds_eye = warp(binary)
        
        if prev_result:
            lanes = find_lanes_with_hint(birds_eye, prev_result.lanes, axis)
        else:
            lanes = find_lanes(birds_eye, axis)
        
        reverse_warped = reverse_warp(birds_eye, undistorted, lanes)
        
        return PipelineResult(original = img, 
                              undistorted = undistorted, 
                              binary = binary, 
                              birds_eye = birds_eye,
                              lanes = lanes,
                              reverse_warped = reverse_warped)
        
    def get_binary_image(self, img):
        return get_binary_image(img, 
                                sobel_kernel = self.sobel_kernel, 
                                thresh = self.thresh)
    
pipeline = Pipeline(mtx, dist, sobel_kernel = 5, thresh = thresholds)
    
# Make a list of test images
images = glob.glob('test_images/*.jpg')

f, axes = plt.subplots(len(images) * 2, 3, figsize=(16, 8 * len(images)))

for idx, fname in enumerate(images):
    top = idx * 2
    bottom = (idx * 2) + 1
    
    result = pipeline.run(fname, None, axes[bottom][0])
    result_hint = pipeline.run(fname, result, axes[bottom][1])
    
    #show_image(axes[idx][0], result.original, False, 'Original')
    show_image(axes[top][0], result.undistorted, False, 'Undistorted')
    show_image(axes[top][1], result.binary, True, 'Binary')
    show_image(axes[top][2], result.birds_eye, True, 'Bird\'s Eye')
    show_image(axes[bottom][2], result.reverse_warped, True, 'Reverse Warped')
    format_lanes(idx, result.lanes, fname)
    format_lanes(idx, result_hint.lanes, fname)

## TODO: Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.